# dbachecks - Saving Configurations

You can alter the configurations for any check at the command line to ensure that you are running checks that will test what you want.

You will want to save those configurations so that you can pass them to other people or run them using automation.

The process that you will need to follow is 

- Create the configuration
- test the configuration
- save the configuration with `Export-DbcConfig`

This will create a json file of the configuration which you can save in source control and then you, others in your team or automation can use the configuration with `Import-DbcConfig`

# Excluding and skipping checks

As well as the configuration items that you saw in the simple configuration notebook, which will change the value for a check such as making sure that the database owners are the accounts that you expect, there are other configuration items which enable you to exclude checks, instances or databases from checks, skip parts of checks.

You can see those configurations by running the block below.

When you are creating configurations for saving and sharing, you will find `command.invokedbccheck.excludecheck` and `command.invokedbccheck.excludedatabases` useful.

`command.invokedbccheck.excludecheck` will set the configuration so that any checks specified here will not run. It is best to exclude Groups of checks rather than individual ones here.
`command.invokedbccheck.excludedatabases` will set the configuration for any databases that you do not wish to run checks against.


In [10]:
Get-DbcConfig | Where-Object{$_.Name -like '*exclude*' -or $_.Name -like '*skip*'} 

# Create a configuration

First you will need to create your configuration. 

## Reset the configuration to default first
It is best practice to run `Reset-DbcConfig` before you start to do this to avoid confusion. This will reset all of the configurations to the default values. It will also list all of the configurations and you can see the current (default) values  

In [11]:
Reset-DbcConfig

## Morning Checks Configuration

Lets create a configuration for some morning checks for our Production estate. You will

- Check the production instances (we only have two in this demo) 'localhost,15592','localhost,15593'
- Ensure the instance is available and the connection authentication scheme is SQL (because we are using SQL Auth) but don't check PSRemoting
- Ensure that all agent jobs have succeeded
- All databases have backups that meet RPO/RTO
- There are no errors in the Error Logs in the last 24 hours
- The disk space is not less than 10% free
- There are no Memory Dumps
- The last duration of the agent jobs was not more than 10% of the average run time including any currently running jobs
- The log file sizes are less than 50% of the data file sizes
- That databases have more than 10% free space available


In [12]:
# Set the instances and computernames to run the check against
Set-DbcConfig -Name app.sqlinstance -Value 'localhost,15592','localhost,15593'
Set-DbcConfig -Name app.computername -Value 'localhost,15592','localhost,15593'
# Set the auth scheme to SQL as we are using SQL Auth
Set-DbcConfig -Name policy.connection.authscheme -Value SQL
# Don't check the PSRemoting
Set-DbcConfig -Name skip.connection.remoting -Value $true
# Set expected backup - 7 days for full - 24 hours for diff - 15 minutes for log
Set-DbcConfig -Name policy.backup.fullmaxdays -Value 7
Set-DbcConfig -Name policy.backup.diffmaxhours -Value 24
Set-DbcConfig -Name policy.backup.logmaxminutes -Value 15
# Set config for disk space
Set-DbcConfig -Name policy.diskspace.percentfree -Value 10
# Maximum number of memory dumps
Set-DbcConfig -Name policy.dump.maxcount -Value 1
# Set config for job run times
Set-DbcConfig -Name agent.lastjobruntime.percentage -Value 10
Set-DbcConfig -Name agent.longrunningjob.percentage -Value 10
# Set max size of log files compared to data file size
Set-DbcConfig -Name policy.database.logfilesizecomparison -Value Average
Set-DbcConfig -Name policy.database.logfilesizepercentage -Value 50
# Set config to ensure files have more than 10% free
Set-DbcConfig -Name policy.database.filegrowthfreespacethreshold -Value 10

# Examining the current configuration and saving it

When you set the configuration, the output shows the cuurently configured values for the configuration items that you have changed.

You can examine the complete current configuration with `Get-DbcConfig`

In [13]:
Get-DbcConfig

## Saving the configuration to a file for reuse

Once you have a configuration created, you can save it for easy importing into your session or for use with automation. I suggest that the best place for you to save this is in a source control repository so changes can be controlled and auditted

To save the current configuration use `Export-DbcConfig`

In [15]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
Export-DbcConfig -Path $FolderPath\Morning-Checks-Production.json 

[09:59:39][Export-DbcConfig] Wrote file to C:\Users\Rob.THEBEARD\Documents\dbachecks\Morning-Checks-Production.json


The file that is created is just json, you can take a look at it (and alter it directly if you wish)

In [16]:
azuredatastudio $FolderPath\Morning-Checks-Production.json 

# Loading the configuration from a file

To use the configuration that you have saved you use `Import-DbcConfig`

This will take the values from the json file and set the configuration accordingly

In [17]:
Import-DbcConfig -Path $FolderPath\Morning-Checks-Production.json 

# Run the Checks with that configuration

You can then run the checks with that configuration as usual. You do not need to specify the computer names or Instance names as they are in the configuration already. You will reset the configuration, load the configuration from the file and run the checks

In [19]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
# Just in case the config has been altered
$null = Reset-DbcConfig
$null = Import-DbcConfig -Path $FolderPath\Morning-Checks-Production.json 
$invokeDbcCheckSplat = @{
    SqlCredential = $SqlCredential
    Check = 'InstanceConnection', 'FailedJob', 'LastBackup', 'ErrorLog', 'DiskCapacity', 'MemoryDump', 'LastJobRunTime', 'LongRunningJob', 'LogfileSize', 'FutureFileGrowth'
}
Invoke-DbcCheck @invokeDbcCheckSplat

## Only showing the failures

This will run 114 checks and most will pass. To avoid filling our buffer will all the checks that have passed, we can only display the failed checks using the `-Show` parameter with the `Fails` value. (NOTE - This is a better option than `Failed` which will only show the failed tests and no context for them)

In [21]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
# Just in case the config has been altered
$null = Reset-DbcConfig
$null = Import-DbcConfig -Path $FolderPath\Morning-Checks-Production.json 
$invokeDbcCheckSplat = @{
    SqlCredential = $SqlCredential
    Check = 'InstanceConnection', 'FailedJob', 'LastBackup', 'ErrorLog', 'DiskCapacity', 'MemoryDump', 'LastJobRunTime', 'LongRunningJob', 'LogfileSize', 'FutureFileGrowth'
    Show = 'Fails'
}
Invoke-DbcCheck @invokeDbcCheckSplat